In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv(r"D:\Dig\SMSSpamCollection",sep="\t",names=['Label','Message'])
data.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data.describe()

,Label,Message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
data.tail(2)

,Label,Message
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [7]:
data.shape

(5572, 2)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label      5572 non-null object
Message    5572 non-null object
dtypes: object(2)
memory usage: 87.2+ KB


In [10]:
data.Label.value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [11]:
# spam:1 & ham:0
data['Label_num']=data.Label.map({'spam':1,'ham':0})

In [12]:
data.head()

,Label,Message,Label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [13]:
X=data.Message
y=data.Label_num

In [14]:
X.shape

(5572,)

In [15]:
y.shape

(5572,)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [20]:
print(X_train.shape)
print(X_test.shape)

(4457,)
(1115,)


In [21]:
print(y_train.shape)
print(y_test.shape)

(4457,)
(1115,)


In [22]:
#Now we will instantiate CountVectorizer obj & then conver our training dataset into documenter matrix
# Why did we split our data into training and testing then efore we instantiate our count vectorizer?
#->The reason is bcz when we anayise our dataset there will be certain words that your model will not able to recognize ,
# so we want to be able to drop those features

from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()

In [23]:
vector.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [25]:
#Convert 1D into 2D therefore we will call transform method
X_train_dtmatrix=vector.transform(X_train)
X_train_dtmatrix.shape

(4457, 7781)

In [26]:
#The above 2 steps would have be done in an single step and that is by using fit_transform
#X_train_val=vector.fit_transform(X_train)
#X_train_val.shape

(4457, 7781)

In [27]:
X_train_dtmatrix

<4457x7781 sparse matrix of type '<class 'numpy.int64'>'
	with 59839 stored elements in Compressed Sparse Row format>

In [28]:
#CountVectorizer object takes 1D obj and transform it to 2D object
X_test_dtmatrix=vector.transform(X_test)
X_test_dtmatrix.shape

(1115, 7781)

In [29]:
#We didnt use fit method before tansform bcz now any new word in our X_test dataset will be droped 

In [30]:
#Now we are ready to build our model....for that we will use MultinomialNiveBase 
from sklearn.naive_bayes import MultinomialNB  

#then we will call fit method to train our model out X_train Documental matrix & y_train datasets

nb_model=MultinomialNB()
nb_model.fit(X_train_dtmatrix,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
y_predict = nb_model.predict(X_test_dtmatrix)

In [32]:
#Now we will compair y_predict with y_test. y_test which is actual result
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_predict))

0.9865470852017937


In [33]:
print(metrics.confusion_matrix(y_test,y_predict))
# 15 elements are not detected properly

[[966   2]
 [ 13 134]]


In [34]:
print(metrics.classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       968
           1       0.99      0.91      0.95       147

    accuracy                           0.99      1115
   macro avg       0.99      0.95      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [35]:
#NOW WE WILL USE LOGISTIC REGRESSION MODEL & COMPAIR IT WITH THE MULTINOMIALNB
from sklearn.linear_model import LogisticRegression
Log_model=LogisticRegression()

In [36]:
Log_model.fit(X_train_dtmatrix,y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
y_log_predict = Log_model.predict(X_test_dtmatrix) 

In [38]:
print(metrics.accuracy_score(y_test,y_log_predict))

0.9856502242152466


In [39]:
print(metrics.confusion_matrix(y_test,y_log_predict))
#16 elements are not detected properly

[[968   0]
 [ 16 131]]


In [40]:
print(metrics.classification_report(y_test,y_log_predict))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       968
           1       1.00      0.89      0.94       147

    accuracy                           0.99      1115
   macro avg       0.99      0.95      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [41]:
#HENCE MULTINOMILANB IS MORE ACCURATE THAN LOGISTICREGRESSION

In [42]:
#How could we tell difference between (FALSEPOSITIVE & FALSENEGATIVE)
#For that : y_test < y_predict 
#for spam & ham it is 1 & 0

In [50]:
# for FALSEPOSITIVE : Actually they are ham predicted as spam
fp = X_test[y_test < y_predict]
print("Actually they are ham predicted as spam :\n",fp)
print("Shape : ",fp.shape)
print("Length : ",len(fp))

Actually they are ham predicted as spam :
 5475    Dhoni have luck to win some big title.so we wi...
3890                    Unlimited texts. Limited minutes.
Name: Message, dtype: object
Shape :  (2,)
Length :  2


In [51]:
# for FALSENEGATIVE : Actually they are spam predicted as ham
fn = X_test[y_test > y_predict]
print("Actually they are spam predicted as ham :\n",fn)
print("Shape : ",fn.shape)
print("Length : ",len(fn))

Actually they are spam predicted as ham :
 3064    Hi babe its Jordan, how r u? Im home from abro...
4298    thesmszone.com lets you send free anonymous an...
1663    Hi if ur lookin 4 saucy daytime fun wiv busty ...
751     Do you realize that in about 40 years, we'll h...
5       FreeMsg Hey there darling it's been 3 week's n...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
68      Did you hear about the new "Divorce Barbie"? I...
2402    Babe: U want me dont u baby! Im nasty and have...
2575    Your next amazing xxx PICSFREE1 video will be ...
1875    Would you like to see my XXX pics they are so ...
955             Filthy stories and GIRLS waiting for your
3419    LIFE has never been this much fun and great un...
2558    This message is brought to you by GMW Ltd. and...
Name: Message, dtype: object
Shape :  (13,)
Length :  13


In [53]:
# LETS FIND OUT MOST COMMON SPAM AND HAM WORDS

X_train_words = vector.get_feature_names()
len(X_train_words)

7781

In [54]:
X_train_words[0:5]

['00', '000', '000pes', '008704050406', '0089']

In [55]:
#feature_count_ : how many times a word or token appeared
nb_model.feature_count_

array([[ 0.,  0.,  1., ...,  1.,  0.,  1.],
       [ 7., 22.,  0., ...,  0.,  1.,  0.]])

In [56]:
#This output means that the 1st token appeared 0 times in ham and 7 times in spam
#To check that :
nb_model.feature_count_.shape

(2, 7781)

In [57]:
#Now if we index this array 1st row will be index 0 and 2nd row will be index 1
#After indexing we will store 1st row in certain variable and 2nd row in certain variable
#And those variable will represend words in ham & words in our spam

In [58]:
ham_word  = nb_model.feature_count_[0,:]
spam_word = nb_model.feature_count_[1,:]

In [59]:
print(ham_word)
print(ham_word.shape)

[0. 0. 1. ... 1. 0. 1.]
(7781,)


In [60]:
print(spam_word)
print(spam_word.shape)

[ 7. 22.  0. ...  0.  1.  0.]
(7781,)


In [61]:
#Now lets combine this 2 together and Create a DataFrame which will represent ham , spam & list of words

tokens = pd.DataFrame({'token':X_train_words , 'Ham':ham_word , 'Spam':spam_word})
tokens.head()

,token,Ham,Spam
0,00,0.0,7.0
1,000,0.0,22.0
2,000pes,1.0,0.0
3,008704050406,0.0,1.0
4,0089,0.0,1.0


In [62]:
#Lets change our index token itself
tokens = tokens.set_index('token')
tokens.head()

,Ham,Spam
token,,
00,0.0,7.0
000,0.0,22.0
000pes,1.0,0.0
008704050406,0.0,1.0
0089,0.0,1.0


In [63]:
tokens.tail()

,Ham,Spam
token,,
zoe,1.0,0.0
zogtorius,1.0,0.0
zoom,1.0,0.0
zouk,0.0,1.0
èn,1.0,0.0


In [64]:
#Class_count : It will tell the number of observations that we have in our ham and spam classes

nb_model.class_count_

array([3857.,  600.])

In [65]:
# Now we will take instansiates of observations & then try to cal the freq of each word in our ham and spam class 
# And once we converted this observations into freq we will cal ham to spam ratio for each of the words 
# And the after that we can dedact which words are most common in spam and which are most common in ham message 

In [68]:
# if we have to calculate the ratios right now we would have to div eg: 1 / 0 and that would give us a ZeroDivisionError
# and in order or avoid that since we have zero both in ham and spam what we will do is to add some num that way we will not run
# into a zero division error

In [69]:
tokens['Ham'] = tokens.Ham + 10
tokens['Spam'] = tokens.Spam + 10

In [70]:
tokens.head()

,Ham,Spam
token,,
00,10.0,17.0
000,10.0,32.0
000pes,11.0,10.0
008704050406,10.0,11.0
0089,10.0,11.0


In [71]:
#Take Ham col div it with its appropriate class count observation and we will convert this num into freq

from __future__ import division

tokens['Ham']=tokens.Ham/nb_model.class_count_[0]
tokens['Spam']=tokens.Spam/nb_model.class_count_[1]

In [72]:
tokens.head()

,Ham,Spam
token,,
00,0.002593,0.028333
000,0.002593,0.053333
000pes,0.002852,0.016667
008704050406,0.002593,0.018333
0089,0.002593,0.018333


In [73]:
tokens['Ham_to_spam_ratio'] = tokens.Ham/tokens.Spam
tokens.head()

,Ham,Spam,Ham_to_spam_ratio
token,,,
00,0.002593,0.028333,0.091507
000,0.002593,0.053333,0.048613
000pes,0.002852,0.016667,0.171117
008704050406,0.002593,0.018333,0.141419
0089,0.002593,0.018333,0.141419


In [83]:
#Those are the most common ham words exists in the 'Ham_to_spam_ratio'
#Arrange it to the decending order 
tokens.sort_values(by=['Ham_to_spam_ratio'],ascending=False).head(10)

,Ham,Spam,Ham_to_spam_ratio
token,,,
my,0.162043,0.035000,4.629801
gt,0.068966,0.016667,4.137931
lt,0.068447,0.016667,4.106819
he,0.054965,0.016667,3.297900
but,0.093337,0.028333,3.294240
ok,0.061706,0.020000,3.085299
me,0.162821,0.055000,2.960379
ll,0.057298,0.020000,2.864921
it,0.151672,0.060000,2.527871


In [82]:
#Those are the most common spam words exists in the 'Ham_to_spam_ratio' check from the bottomaz DDC
#Arrange it to the decending order 
tokens.sort_values(by=['Ham_to_spam_ratio'],ascending=False).tail(10)

,Ham,Spam,Ham_to_spam_ratio
token,,,
mobile,0.005704,0.191667,0.029760
tone,0.002593,0.093333,0.027779
co,0.002593,0.093333,0.027779
nokia,0.002852,0.105000,0.027161
txt,0.005704,0.226667,0.025164
150p,0.002593,0.103333,0.025091
uk,0.002852,0.116667,0.024445
prize,0.002593,0.133333,0.019445
www,0.002852,0.151667,0.018804


In [85]:
tokens['Spam_to_Ham_ratio'] = tokens.Spam/tokens.Ham
tokens.head()

,Ham,Spam,Ham_to_spam_ratio,Spam_to_Ham_ratio
token,,,,
claim,0.002593,0.175000,0.014815,67.497500
www,0.002852,0.151667,0.018804,53.179848
prize,0.002593,0.133333,0.019445,51.426667
uk,0.002852,0.116667,0.024445,40.907576
150p,0.002593,0.103333,0.025091,39.855667


In [86]:
#Those are the most common spam words exists in the 'Spam_to_ham_ratio'
#Arrange it to the decending order 
tokens.sort_values(by=['Spam_to_Ham_ratio'],ascending=False).head(10)

,Ham,Spam,Ham_to_spam_ratio,Spam_to_Ham_ratio
token,,,,
claim,0.002593,0.175000,0.014815,67.497500
www,0.002852,0.151667,0.018804,53.179848
prize,0.002593,0.133333,0.019445,51.426667
uk,0.002852,0.116667,0.024445,40.907576
150p,0.002593,0.103333,0.025091,39.855667
txt,0.005704,0.226667,0.025164,39.738788
nokia,0.002852,0.105000,0.027161,36.816818
co,0.002593,0.093333,0.027779,35.998667
tone,0.002593,0.093333,0.027779,35.998667
